In [6]:
import pandas as pd
import requests
import time
from datetime import datetime, timedelta
import coinbase



In [40]:

def fetch_historical_data(base_curr, quote_curr, start_time, end_time, granularity=86400):
    """
    Fetches all available historical OHLCV data from Coinbase API.
    
    Args:
    - base_curr (str): Cryptocurrency base (e.g., "BTC", "ETH").
    - quote_curr (str): Cryptocurrency base (e.g., "USDT").
    - start_time (str): Start date in "YYYY-MM-DD" format.
    - end_time (str): End date in "YYYY-MM-DD" format.
    - granularity (int): Timeframe in seconds (default: 86400 for daily data).
    
    Returns:
    - pd.DataFrame: Dataframe containing all historical OHLCV data.
    """

    # Convert symbol format (BTCUSDT -> BTC-USD)
    pair = f"{base_curr}-{quote_curr}"

    url = f"https://api.exchange.coinbase.com/products/{pair}/candles"
    
    # Convert start_time and end_time to datetime objects
    start_dt = datetime.strptime(start_time, "%Y-%m-%d")
    end_dt = datetime.strptime(end_time, "%Y-%m-%d")

    all_data = []
    
    while start_dt < end_dt:
        batch_end = min(end_dt, start_dt + timedelta(seconds=granularity * 300))
        
        params = {
            "granularity": granularity,
            "start": start_dt.strftime("%Y-%m-%dT%H:%M:%SZ"),
            "end": batch_end.strftime("%Y-%m-%dT%H:%M:%SZ")
        }

        response = requests.get(url, params=params)
        
        if response.status_code != 200:
            print(f"Error: {response.status_code}, {response.text}")
            break

        data = response.json()
        
        if not data:
            print("No data found for this period.")
            break
        
        all_data.extend(data)
        
        # Move start time forward to continue fetching
        start_dt = batch_end + timedelta(seconds=granularity)

    if not all_data:
        print("No historical data available.")
        return None

    # Convert to DataFrame
    df = pd.DataFrame(all_data, columns=["timestamp", "low", "high", "open", "close", "volume"])
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")
    df = df.sort_values("timestamp").reset_index(drop=True)

    return df

# Example Usage
df = fetch_historical_data("BTC", "USDT", "2024-01-01", "2024-03-01")
print(df.head())


   timestamp       low      high      open     close       volume
0 2024-01-01  42178.40  44195.57  42293.69  44176.97   729.393558
1 2024-01-02  44157.45  45880.95  44183.08  44954.64  1767.612268
2 2024-01-03  40545.71  45507.46  44959.03  42841.60  2638.324645
3 2024-01-04  42619.10  44745.07  42841.76  44154.57  1386.935236
4 2024-01-05  42401.97  44368.38  44153.69  44128.22  1361.430971


In [41]:
df

,timestamp,low,high,open,close,volume
0,2024-01-01,42178.40,44195.57,42293.69,44176.97,729.393558
1,2024-01-02,44157.45,45880.95,44183.08,44954.64,1767.612268
2,2024-01-03,40545.71,45507.46,44959.03,42841.60,2638.324645
3,2024-01-04,42619.10,44745.07,42841.76,44154.57,1386.935236
4,2024-01-05,42401.97,44368.38,44153.69,44128.22,1361.430971
...,...,...,...,...,...,...
56,2024-02-26,50899.03,54919.40,51741.27,54487.63,912.949690
57,2024-02-27,54461.08,57606.19,54475.80,57040.70,1781.906027
58,2024-02-28,56696.69,64075.18,57042.88,62442.40,2221.691972
59,2024-02-29,60367.33,63657.19,62427.46,61148.02,1827.605099
